## Week 8. Multi Layer Perceptron

기계학습 8주차는 세가지 목표가 있다.
1. sigmoid with regression & k-fold validation
2. Perceptron
3. Multi Layer Perceptron (MLP)

## 0. Load data

이번 실습은 다시 분류 문제를 해결한다.  
이번 주는 먼저 회귀방법으로 분류 문제를 푸는 기법을 소개하겠다.  
dataset covertype이며 인공위성이 찍은 사진을 전처리하여 table data로 작성한 데이터셋이다.  
4.4만개의 instance가 54의 feature을 가지고 있다. 

In [1]:
import numpy as np
import pandas as pd

x = pd.read_csv('08_mlp_x_train.csv')
y = pd.read_csv('08_mlp_y_train.csv')

x.shape, y.shape

((44267, 54), (44267, 1))

## 1. Logistic regression

Logistic regression은 linear regression method를 분류 문제에 적용한 방법이다.  
Regression 모델의 output은 continous value이다.  
continous value를 분류문제로 적용하기 위해 output이 특수한 함수를 거치도록 설계되었다.  
함수 sigmoid는 continuous value를 0 ~ 1 사이의 probability로 변환시켜준다.   

In [2]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

모델의 평가를 위해 이번 실습부터 K-fold를 사용하겠다.  
K-fold는 train set을 k등분하여 교차검증하는 방법이다.  
트레이닝 셋을 k개의 Fold를 나눠서 그중 1개씩 뽑아서 k번 split돌려서 학습시킨다.  

Train data중 일부만 추출해내어 model의 성능을 평가하는 것은 추출된 sample에 따라 편향될 가능성이 존재한다.  
K-Fold는 모든 train sample에 대해 예측하고 성능을 평가했기 대문에 model성능의 일관성 또한 확인할 수 있다.  
때문에 모든 train sample을 설명할 수 있는 model을 만든다면 test data도 잘 예측할 수 있다는 기대를 할 수 있다.  
안정된 모델을 만든다.  

In [4]:
from sklearn.model_selection import KFold
kf = KFold(shuffle = True)

test_index_set = set()  # set 타입은 중복된 값이 들어갈 수 없다.  

for train_index, test_index in kf.split(x,y):
    test_index_set.update(test_index)
    
print(len(test_index_set), x.shape[0])   # 테스트 셋을 전부 교차 검증한다.

44267 44267


이제 Logistic regression을 5 fold로 교차검증하겠다.  
기존 학습에 걸리던 시간보다 K배의 시간이 들지만 모델 성능을 범용적으로 평가할 수 있다.  
지금부터 코드 결과가 느리게 나온다. 

In [5]:
import warnings
warnings.filterwarnings('ignore')

scores = []
logistic = LogisticRegression(max_iter = 100)
for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test = \
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    logistic.fit(x_train, y_train.values.ravel())
    scores.append(logistic.score(x_test, y_test.values.ravel()))

print(f'평균 정확도 : {np.array(scores).mean()*100:.5}%, 정확도의 편차 : {np.array(scores).std():.3}')

평균 정확도 : 72.517%, 정확도의 편차 : 0.00518


## 2. Perceptron

우리는 선형회귀를 위해 coefficients 베타 또는 weights w를 계산하였다.  
미분방정식을 풀어 error를 최소화하는 w를 찾았고 구한 w는 어느정도 실제 target을 설명할 수 있다.  
Perceptron은 w를 수학적 해로 구하는 것이 아니라 error를 최소화 하도록 반복하여 개선시키는 방법이다.  
두 클래스를 양분하는 선을 찾기위해 반복해가며 error를 최소화 하는 방향으로 w를 개선시킨다.  
이 때문에 시간이 많이 걸린다.  

현재의 w를 사용하여 data를 입력했을 때 예측하는 과정을 feed - forward라고 한다.  
그리고 실제 y와 예측된 Y^의 차를 가지고 weights w를 업데이트 하는 과정을 back - propagation이라고 한다.

In [6]:
from sklearn.linear_model import Perceptron

percep = Perceptron(n_jobs = -1, eta0 = 0.0001,   # wegiht를 얼마만큼 바꿀건지, 작게하면 에러를 100만큼 발생했지만 1만큼만 반영함
                                                    # 델타 t를 점진적으로 증가시킴 
                   max_iter = 10000,
                    # verbose = 1,  # 과정 보이게 하는거 
                   tol = 1e-5, n_iter_no_change = 100)

scores = []
for train_index, test_index in kf.split(x,y):
    x_train, x_test, y_train, y_test = \
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    percep.fit(x_train, y_train.values.ravel())
    scores.append(percep.score(x_test, y_test.values.ravel()))
print(scores)
print(f'평균 정확도 : {np.array(scores).mean()*100:.5}%, 정확도의 편차 : {np.array(scores).std():.3}')

[0.6796927942173029, 0.6773209848655974, 0.7173839376482548, 0.6962611544109342, 0.6523212470349035]
평균 정확도 : 68.46%, 정확도의 편차 : 0.0216


복잡한 dimension을 관통하는 경계를 제안하여 두 class를 양분한다는건 어려운 일이다.  
수학적인 해를 구하는 것 뿐만 아니라 error를 반영하여 개선하는 방법도 기계학습 방법 중 하나로 사용할 수 있음을 boosting 실습때 확인했다.  
그런데 perceptron은 error를 개선하는데 한계에 부딧혀 test set을 분류하는 정확도를 일정 이상 개선하지 못했다.  

## 3. Multi Layer Perceptron

MLP는 perceptron을 병렬화, 복층화하여 만든 모델이다.  
여러 perceptron을 동시에 개선하며, 복층 구조로 인해 XOR문제를 해결할 수 있게 되었다.  
하나의 perceptron을 사용할경우 XOR문제와 더 복잡한 data를 분리하는 선을 그을수없지만   
perceptron output들을 모아 다시한번 perceptron을 구성함으로써 XOR문제를 해결할 수 있게 된다.  
그리고 Layer의 깊이가 더 깊어질수록 다양한 경계를 그을 수 있다고 설명한다.  
이번과제는 시간이 너무 오래걸리니까 시각화에 압박감을 느끼지 말아달라 

In [7]:
from sklearn.neural_network import MLPClassifier

MLPClassifier()

MLPClassifier()

In [9]:
%%time
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier((100,),
                   activation = 'identity',
                   max_iter = 200,
                   n_iter_no_change = 10,
                   )

scores = []
for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test = \
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    mlp.fit(x_train, y_train.values.ravel())
    scores.append(mlp.score(x_test, y_test.values.ravel()))
print(scores)
print(f'평균 정확도 : {np.array(scores).mean()*100:.5}%, 정확도의 편차 : {np.array(scores).std():.3}')

[0.6752880054212785, 0.7367291619606957, 0.6916299559471366, 0.7499152829549305, 0.7184005421890884]
평균 정확도 : 71.439%, 정확도의 편차 : 0.0277
Wall time: 11.2 s


In [10]:
%%time
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier((100,100),
                   activation = 'identity',
                   max_iter = 200,
                   n_iter_no_change = 10,
                   )

scores = []
for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test = \
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    mlp.fit(x_train, y_train.values.ravel())
    scores.append(mlp.score(x_test, y_test.values.ravel()))
print(scores)
print(f'평균 정확도 : {np.array(scores).mean()*100:.5}%, 정확도의 편차 : {np.array(scores).std():.3}')

[0.7371809351705444, 0.7254348317144793, 0.7272111148763131, 0.7059753755788998, 0.6340223652998983]
평균 정확도 : 70.596%, 정확도의 편차 : 0.0374
Wall time: 2min 5s


In [11]:
%%time
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier((100,100,100),
                   activation = 'identity',
                   max_iter = 200,
                   n_iter_no_change = 10,
                   )

scores = []
for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test = \
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    mlp.fit(x_train, y_train.values.ravel())
    scores.append(mlp.score(x_test, y_test.values.ravel()))
print(scores)
print(f'평균 정확도 : {np.array(scores).mean()*100:.5}%, 정확도의 편차 : {np.array(scores).std():.3}')

[0.7026202846171222, 0.7339055793991416, 0.7247260815542754, 0.7468654693324297, 0.7197560149101999]
평균 정확도 : 72.557%, 정확도의 편차 : 0.0147
Wall time: 59.4 s


층이 깊어질수록 다양한 경계를 생성할 수 있게된다.  
또한 시간도 줄어즈는것을 보게 되었다. 
제시된 경계가 세밀해짐에 따라 모델의 성능이 향상되지만 overfit을 발생시킨다. 
Overfit으로 인해 우리가 모델을 적용하기 원하는 test set에 대해서 성능이 나빠질수있다.  
또한 overfit과 별개로 학습시간이 증가하기 때문에 perceptron의 갯수를 무조껀 늘리는 것은 바람직하지 않다.  

## 5. Conclusion

선형 회귀를 사용한 분류 기법 logistics regression을 실습하였다.  
기법은 선형회귀의 출력값을 function sigmoid에 통과시켜 확률 output을 갖도록 변형시켰고 분류 문제에 풀수있도록 고안됬다. 
error를 최소화하는 베타를 계산하여 sample들을 성공적으로 분류했다.  
반대로 베타를 무작위로 생성한 뒤 iterative하게 개선시키는 perceptron도 실습하였다.  
perceptron은 y^과 y의 차이를 줄여나가 y^가 y와 일치하도록 W를 개선하였다.  
하지만 XOR문제를 해결할수없었고 점진적으로 개선한 w가 최적의 w여부는 장담할수없었다.  
MLP의 층이 늘어날수록 다양한 경계를 제시할 수 있다.  
하지만 다양한 경계는 overfit을 불러일으키며 층이 늘어날수록 학습시간 또한 증가하였다.  